In [4]:
from ast import literal_eval
import pandas as pd

df = pd.read_csv('dataset.csv', converters={'paragraphs': literal_eval})
df.head()

,question_text,short_answers,title,paragraphs
0,which is the most common use of opt-in e-mail ...,a newsletter sent to an advertising firm 's cu...,Email marketing,[Email marketing - Wikipedia Email marketing ...
1,how i.met your mother who is the mother,Tracy McConnell,The Mother ( How I Met Your Mother ),[The Mother ( How I Met Your Mother ) - wikipe...
2,who had the most wins in the nfl,Tom Brady,List of National Football League career quarte...,[List of National Football League career quart...
3,who played mantis guardians of the galaxy 2,Pom Klementieff,Pom Klementieff,[Pom Klementieff - wikipedia Pom Klementieff ...
4,the nashville sound brought a polished and cos...,the use of lush string arrangements with a rea...,Nashville sound,[Nashville sound - wikipedia Nashville sound ...


In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

text = df['paragraphs'][3]

questions = [
    "who played mantis guardians of the galaxy 2",
]

for question in questions:
    inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer_start_scores, answer_end_scores = model(**inputs)

    answer_start = torch.argmax(
        answer_start_scores
    )  # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

In [ ]:

tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")

model = AutoModelForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")
text = df['paragraphs'][3]

questions = [
    "who played mantis guardians of the galaxy 2",
]

for question in questions:
    inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer_start_scores, answer_end_scores = model(**inputs)

    answer_start = torch.argmax(
        answer_start_scores
    )  # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    print(f"Question: {question}")
    print(f"Answer: {answer}\n")


# Pipeline

QuestionAnsweringPipeline
classtransformers.QuestionAnsweringPipeline(model: Union[PreTrainedModel, TFPreTrainedModel], tokenizer: transformers.tokenization_utils.PreTrainedTokenizer, modelcard: Optional[transformers.modelcard.ModelCard] = None, framework: Optional[str] = None, device: int = - 1, task: str = '', **kwargs)[SOURCE]
Question Answering pipeline using ModelForQuestionAnswering head. See the question answering usage examples for more information.

This question answering can currently be loaded from the pipeline() method using the following task identifier(s):

“question-answering”, for answering questions given a context.

The models that this pipeline can use are models that have been fine-tuned on a question answering task. See the up-to-date list of available models on huggingface.co/models.

Parameters
model (PreTrainedModel or TFPreTrainedModel) – The model that will be used by the pipeline to make predictions. This needs to be a model inheriting from PreTrainedModel for PyTorch and TFPreTrainedModel for TensorFlow.

tokenizer (PreTrainedTokenizer) – The tokenizer that will be used by the pipeline to encode data for the model. This object inherits from PreTrainedTokenizer.

modelcard (str or ModelCard, optional, defaults to None) – Model card attributed to the model for this pipeline.

framework (str, optional, defaults to None) –

The framework to use, either “pt” for PyTorch or “tf” for TensorFlow. The specified framework must be installed.

If no framework is specified, will default to the one currently installed. If no framework is specified and both frameworks are installed, will default to PyTorch.

args_parser (ArgumentHandler, optional, defaults to None) – Reference to the object in charge of parsing supplied pipeline parameters.

device (int, optional, defaults to -1) – Device ordinal for CPU/GPU supports. Setting this to -1 will leverage CPU, >=0 will run the model on the associated CUDA device id.

In [2]:
# from transformers import pipeline

# nlp = pipeline("question-answering")

# context = r"""
# Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a
# question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune
# a model on a SQuAD task, you may leverage the `run_squad.py`.
# """

# print(nlp(question="What is extractive question answering?", context=context))
# print(nlp(question="What is a good example of a question answering dataset?", context=context))

In [1]:
from transformers import pipeline

nlp_bert = pipeline("question-answering")

In [5]:
print("Question: ", df['question_text'][3])
print("Annotated answer: ", df['short_answers'][3])
print(nlp(question=df['question_text'][3], context=df['paragraphs'][3]))

Question:  who played mantis guardians of the galaxy 2
Annotated answer:  Pom Klementieff


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 5426.01it/s]


{'score': 0.0035539168985547764, 'start': 0, 'end': 15, 'answer': 'Pom Klementieff'}


In [6]:
print("Question: ", df['question_text'][1])
print("Annotated answer: ", df['short_answers'][1])

print(nlp(question="What is the mother in how i met your mother?", context="".join(df['paragraphs'][1])))

Question:  how i.met your mother who is the mother
Annotated answer:  Tracy McConnell


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 5236.33it/s]


{'score': 0.4352570122007826, 'start': 50, 'end': 60, 'answer': 'The Mother'}


In [12]:
from transformers.pipelines import pipeline
from transformers.modeling_auto import AutoModelForQuestionAnswering
from transformers.tokenization_auto import AutoTokenizer

#model_name = "deepset/roberta-base-squad2"

model_name = "elgeish/cs224n-squad2.0-albert-base-v2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
print(nlp(question="What is the mother in how i met your mother?", context="".join(df['paragraphs'][1])))

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 4056.39it/s]


KeyboardInterrupt: 

In [ ]:
model_name = "xlnet-base-cased"
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
print(nlp(question="What is the mother in how i met your mother?", context="".join(df['paragraphs'][1])))

In [ ]:
model_name = "bert-base-uncased"
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
print(nlp(question="What is the mother in how i met your mother?", context="".join(df['paragraphs'][1])))

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 5140.08it/s]
